# Bayesian Nash Equilibrium in double bi-matrix games

This notebook introduces a solution method suggested by William Spaniel for analyzing bimatrix games where one of the players can have multiple types in this video: https://youtu.be/E0_CA9TwZ8c. It is recommended to check the video out in order to fully understand how and why the method works. This code solve BNE for an ex ante Bayesian game.

In [2]:
import pandas as pd 
import numpy as np 
import itertools
import nashpy
import bimatrix

Player types 
* Player 1: always a stag hunt type, 
* Player 2: with prob. `p`, she is PD, and `1-p`, she is SH. 

Payoffs: a payoff matrix *list* for each player: one payoff matrix for each type that player 2 can have. 

In [3]:
def compute_full_matrix(U1, U2, p, action_names=None): 
    '''
        Assumes that only player 2's type varies 
        (this means that player 1 has one action per row in U1, 
         while 2 has nA2**2 (one choice per type))
        Both players have one utility matrix for each realization 
        of player 2's type. 
         
        INPUTS: 
            U1: list of 2 payoff matrices for player 1 (row player)
            U2: list of 2 payoff matrices for player 2 (column player)
            p: (scalar) Probability that player 2 is the first type 
            action_names: [optional] 2-list of names of actions (nA1 and nA2 long)
        OUTPUTS: 
            t1, t2: wide-form payoff matrices suitable for finding the NE 
            A1, A2: names of actions 
    '''
    assert len(U1) == 2
    assert len(U2) == 2
    assert np.isscalar(p)
    nA1, nA2 = U1[0].shape  # Number of actions for player 1 and 2
    
    t1 = np.empty((nA1, nA2*nA2))
    t2 = np.empty((nA1, nA2*nA2))
    
    # player 1 chooses an action without knowing what type 2 is 
    for ia1 in range(nA1): 
        i_col = 0 
        
        # player 2 chooses an action conditional on observing her type 
        for a2_1 in range(nA2): 
            for a2_2 in range(nA2): 
                # Ex-ante
                t1[ia1,i_col] = p * U1[0][ia1,a2_1] + (1.-p) * U1[1][ia1,a2_2]
                t2[ia1,i_col] = p * U2[0][ia1,a2_1] + (1.-p) * U2[1][ia1,a2_2]
                
                i_col += 1
                
    if action_names is None: 
        A1 = [f'{i}' for i in range(nA1)]
        A2 = [f'{a}{b}' for a in range(nA2) for b in range(nA2)]
    else: 
        assert len(action_names) == 2 
        A1 = action_names[0]
        assert len(A1) == nA1, f'Incorrect # of action names'
        a2 = action_names[1]
        assert len(a2) == nA2, f'Incorrect # of action names'
        
        A2 = [f'{a}{b}' for a in a2 for b in a2]
        
    return t1, t2, A1, A2

# Spaniel's example

* Player 1 just has one type and receives Stag Hunt payoffs regardless 
* Player 2 can be two types: Prisoner's Dilemma type (probability 0.2) or Stag Hunt type (probability 0.8).

Ex Ante Bayesian game formulation, $\Gamma = (\mathcal{N}, (\mathcal{T}_n)_n, (\mathcal{A}_n)_n, (u_n)_n, (p(\cdot\,|\,t_n))_{t_n, n}, (p(t_n))_{t_n, n})$, where
* Player set, $\mathcal{N}=\{$player 1, player 2$\}$
* Type set, $\mathcal{T}_1=\{$Stag hunt$\}$, $\mathcal{T}_2 = \{$ Prisoner's dilemma, Stag hunt$\}$
* Action space, $\mathcal{A}_1 = \{$U, D$\}$, $\mathcal{A}_2 = \{$L, R$\}$
* Utility, $u_1(t_1, t_2, a_1, a_2)\in \mathbb{R}$, $u_2(t_1, t_2, a_1, a_2) \in \mathbb{R}$
* Belief system
\begin{align*}
p(t_1 = \text{Stag hunt}\,|\, t_2 = \text{Stag hunt})=1 &, p(t_1=\text{Stag hunt}\,|\,t_2 = \text{Prisoner's dilemma})=1\\
p(t_1 = \text{Stag hunt}\,|\, t_2 = \text{Stag hunt})=0.8 &, p(t_1=\text{Prisoner's dilemma}\,|\,t_2 = \text{Stag hunt})=0.2
\end{align*}
* Prior distribution over types, 
\begin{align*}
p(t_1=\text{Stag hunt})&=1\\
p(t_2 = \text{Stag hunt})&=0.8, p(t_2 = \text{Prisoner's dilemma})=0.2
\end{align*}
# Ex ante expected utility (General form)
$U_n(\pi_n, \pi_{-n}) = \sum_{t_n \in \mathcal{T}_n}p(t_n) \sum_{t_{-n}\in \mathcal{T}_{-n}} p(t_{-n}\,|\,t_n) \sum_{a\in \mathcal{A}} \left(\prod_{i\in\mathcal{N}}\pi_i(t_i)(a_i)\right)u_n(t_n, t_{-n}, a_n, a_{-n})$

# Ex ante Bayesian Nash Equilibrium
The strategy profile $\pi^*$ is an ex ante Bayesian Nash equilibrium if $U_n(\pi_n^*, \pi_{_n}^*)\geq U_n(\pi_n, \pi_{-n}^*)$ for all $\pi_n$ and all $n$, where $\pi_i:\mathcal{T}_i$ (characterizes the information set in extensive game form) $\mapsto \Delta(\mathcal{A}_i)$

In [4]:
# Pr(player 2 is the PD type)
p = 0.2
p_distri = {'PD_2':p, 'SH_2':1-p}
prior_2 = p_distri

# Pr(player 1 is the SH type)
q = 1.0
q_distri = {'SH_1':q}
prior_1 = q_distri 

# player 1 
u11  = np.array([[3,0], [2,1]])
# U1 = [u11, u11] # player 1 has same payoffs regardless of 2's type
U11 = [u11, u11]
a1 = ['U','D']
A1 = [f'{a}' for a in a1]
# A1 = ['U', 'D']

T1 = ['SH_1']
U1 = [u11]

# player 2
u21 = np.array([[3,4],[1,2]]) # prisoner's dilemma (player 2 type 1)
u22 = np.array([[3,2],[0,1]]) # stag hunt (player 2 type 2)
U2 = [u21, u22]
a2 = ['L', 'R']
A2 = [f'{a}{b}' for a in a2 for b in a2]


T2 = ['PD_2', 'SH_2']

The following two tables are tables of $\sum_{a\in \mathcal{A}} \left(\prod_{i\in\mathcal{N}}\pi_i(t_i)(a_i)\right)u_n(t_n, t_{-n}, a_n, a_{-n})$ for both players $n$ under all pure strategy $(\pi_n(t_n), \pi_{-n}(t_{-n}))$ with all pairs of $(t_n , t_{-n})$.

* Here, all pairs of $(t_n, t_{-n})$: $(t_{11}, t_{21}), (t_{11}, t_{22})$

* Therefore, we have two payoff matrix systems.

In [5]:
print(f'--- If P2 is type 0 ---')
bimatrix.print_payoffs([u11, u21], [a1, a2])

--- If P2 is type 0 ---


,L,R
U,"(3, 3)","(0, 4)"
D,"(2, 1)","(1, 2)"


In [6]:
print(f'--- If P2 is type 1 ---')
bimatrix.print_payoffs([u11, u22], [a1, a2])

--- If P2 is type 1 ---


,L,R
U,"(3, 3)","(0, 2)"
D,"(2, 0)","(1, 1)"


## Wide form
We first convert the game to wide matrix form 

In [7]:
 t1, t2, A1, A2 = compute_full_matrix(U11, U2, p, [A1, a2])

In [8]:
bimatrix.print_payoffs([t1, t2], [A1,  A2], 3)

,LL,LR,RL,RR
U,"(3.0, 3.0)","(0.6, 2.2)","(2.4, 3.2)","(0.0, 2.4)"
D,"(2.0, 0.2)","(1.2, 1.0)","(1.8, 0.4)","(1.0, 1.2)"


## Removing strictly dominated strategies

Looking for strictly dominated strategies: we know that if P2 is the Prisonner's Dilemma type, playing $R$ should be a strictly dominating strategy (defecting). And this comes out of running IESDS on the wide-form matrix representation of the game. 

In [9]:
A_, T_ = bimatrix.IESDS(A=[A1, A2], U=[t1, t2], DOPRINT=True)

D[0] False
D[1] True
D[0] False
D[1] False


Since those actions were strictly dominated, it suffices to focus on this reduced version of the game. 

In [10]:
bimatrix.print_payoffs(T_, A_, 3)

,RL,RR
U,"(2.4, 3.2)","(0.0, 2.4)"
D,"(1.8, 0.4)","(1.0, 1.2)"


## Solving 
Now we simply call a game theory solver to find all equilibria of the game. 

In [11]:
eqs = list(nashpy.Game(T_[0], T_[1]).support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

Found 3 equilibria
1: s1 = [1. 0.], s2 = [1. 0.]
2: s1 = [0. 1.], s2 = [0. 1.]
3: s1 = [0.5 0.5], s2 = [0.625 0.375]


## A warning: calling `nashpy` on the full game

Interestingly, it  seems that the nashpy `support_enumeration` is getting confused if we provide it the full game as opposed to eliminating strictly dominated strategies beforehand. 

In [12]:
G = nashpy.Game(t1, t2)

eqs = list(G.support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i,eq in enumerate(eqs): 
    print(f'{i+1}: s1 = {eq[0]}, s2 = {eq[1]}')

Found 2 equilibria
1: s1 = [1. 0.], s2 = [0. 0. 1. 0.]
2: s1 = [0. 1.], s2 = [0. 0. 0. 1.]


C:\Users\12646\.conda\envs\game\Lib\site-packages\nashpy\algorithms\support_enumeration.py:259: RuntimeWarning: 
An even number of (2) equilibria was returned. This
indicates that the game is degenerate. Consider using another algorithm
to investigate.
                  
  warnings.warn(warning, RuntimeWarning)


# Test Codes (scratch)

In [13]:
def ex_ante_exp_comp(U1, U2, A1, A2, a1, a2, prior_2, prior_1, T1, T2, action_names=None): 
    '''
        Assumes that only player 2's type varies 
        (this means that player 1 has one action per row in U1, 
         while 2 has nA2**2 (one choice per type))
        Both players have one utility matrix for each realization 
        of player 2's type. 
         
        INPUTS: 
            U1: list of 2 payoff matrices for player 1 (row player)
            U2: list of 2 payoff matrices for player 2 (column player)
            p: (scalar) Probability that player 2 is the first type 
            action_names: [optional] 2-list of names of actions (nA1 and nA2 long)
            T1: list of player 1 types
            T2: list of player 2 types
        OUTPUTS: 
            payoff1, payoff2: wide-form payoff matrices suitable for finding the NE 
            A1, A2: names of actions 
    '''
    # 2-player ex Ante Bayesian game formulation
    
    # Check the prior distribution are valid
    for item in prior_1:
        assert np.isscalar(prior_1[item])
        assert prior_1[item] <= 1.0
        assert prior_1[item] >= 0.0
        
    for item in prior_2:
        assert np.isscalar(prior_2[item])
        assert prior_2[item] <= 1.0
        assert prior_2[item] >= 0.0
        
    prior_list = [prior_1, prior_2]
    
    num_a1, num_a2 = U1[0].shape    # Number of actions for player 1 and 2
    num_T1, num_T2 = len(T1), len(T2)   # Number of types for player 1 and 2
    
    # Initialize wide payoff matrix 
    payoff1 = np.empty((num_a1*num_T1, num_a2*num_T2))
    payoff2 = np.empty((num_a1*num_T1, num_a2*num_T2))
    # print('payoff1', payoff1)
    # print('payoff2', payoff2)
        
    # Construct self-type conditional belief of other player types, p( | t_n)
    belief_construct(T1, T2, prior_1, prior_2)
    belief_construct(T2, T1, prior_2, prior_1)
    
    # Compute interim expected utility for player 1
    interim_exp_comp(cond_player='player 1', U_cond=U1, T_cond=T1, T_targ=T2, a_cond=a1, a_targ=a2, A_targ=A2)
    # Compute interim expected utility for player 2
    interim_exp_comp(cond_player='player 2', U_cond=U2, T_cond=T2, T_targ=T1, a_cond=a2, a_targ=a1, A_targ=A1)
    
    for i_row, strgy_1 in enumerate(A1):
        for i_col, strgy_2 in enumerate(A2):
            holder1 = []
            holder2 = []
            # Ex-ante expected utility for player 2
            for indx_t2, t2 in enumerate(T2):
                holder2.append(prior_2[t2] * interim_exp[t2][(strgy_2[indx_t2], strgy_1)])
            for indx_t1, t1 in enumerate(T1):
                holder1.append(prior_1[t1] * interim_exp[t1][(strgy_1[indx_t1], strgy_2)])
            
            payoff1[i_row, i_col] = sum(holder1)
            payoff2[i_row, i_col] = sum(holder2)
    
    A1_indx, A2_indx = None, None
    if action_names is not None:
        action_name_1 = action_names[0]
        action_name_2 = action_names[1]
    else:
        action_name_1, action_name_2 = None, None
       
    for count in range(len(T1)):
        A1_indx = two_type_strgy_name_gen(A=A1_indx, action_name=action_name_1, num_a=num_a1)
    for count in range(len(T2)):
        A2_indx = two_type_strgy_name_gen(A=A2_indx, action_name=action_name_2, num_a=num_a2)
        
    return payoff1, payoff2, A1_indx, A2_indx

In [25]:
t1_mod, t2_mod, A1_mod, A2_mod = ex_ante_exp_comp(U1, U2, A1, A2, a1, a2, p_distri, q_distri, T1, T2, action_names=[A1, a2])
bimatrix.print_payoffs([t1_mod, t2_mod], [A1_mod, A2_mod], 3)

,LL,LR,RL,RR
U,"(3.0, 3.0)","(0.6, 2.2)","(2.4, 3.2)","(0.0, 2.4)"
D,"(2.0, 0.2)","(1.2, 1.0)","(1.8, 0.4)","(1.0, 1.2)"


In [26]:
t2_mod

array([[3. , 2.2, 3.2, 2.4],
       [0.2, 1. , 0.4, 1.2]])

In [27]:
A_, T_ = bimatrix.IESDS(A=[A1_mod, A2_mod], U=[t1_mod, t2_mod], DOPRINT=True)
bimatrix.print_payoffs(T_, A_, 3)

D[0] False
D[1] True
D[0] False
D[1] False


,RL,RR
U,"(2.4, 3.2)","(0.0, 2.4)"
D,"(1.8, 0.4)","(1.0, 1.2)"


# Solve the nash equilibria
Nash equilibria is strategy profile $(\pi^*_1, \pi^*_2)$, where $\pi^*_1:\mathcal{T}_1\mapsto\Delta(\mathcal{A}_1)$, $\pi^*_2:\mathcal{T}_2\mapsto\Delta(\mathcal{A}_2)$

From nashpy, with $\mathcal{A}_1 =\{$'U', 'D'$\}$, $\mathcal{A}_2=\{$'L', 'R'$\}$, the equilibria are:
* $\pi^*_1(t_{11})=[1, 0]$, and $\pi^*_2(t_{21})=[0, 1]$, $\pi^*_2(t_{22})=[1, 0]$
* $\pi^*_1(t_{11})=[0, 1]$, and $\pi^*_2(t_{21})=[0, 1]$, $\pi^*_2(t_{22})=[0, 1]$
* $\pi^*_1(t_{11})=[0.5, 0.5]$, and $\pi^*_2(t_{21})=[0, 1]$, $\pi^*_2(t_{22})=[0.625, 0.375]$
# Question: What if the strategy $\pi^*_2(t_{21})$ is not deterministic? Given nashpy result, what would be $\pi^*_2$?

In [17]:
eqs = list(nashpy.Game(T_[0], T_[1]).support_enumeration())
print(f'Found {len(eqs)} equilibria')
for i, eq in enumerate(eqs):
    print(f'{i}: strgy1={eq[0]}, strgy2={eq[1]}')

Found 3 equilibria
0: strgy1=[1. 0.], strgy2=[1. 0.]
1: strgy1=[0. 1.], strgy2=[0. 1.]
2: strgy1=[0.5 0.5], strgy2=[0.625 0.375]


In [18]:
def interim_exp_comp(cond_player, U_cond, T_cond, a_cond, a_targ, A_targ, T_targ):
    # Compute interim expected utility under all pure strategies (2 player)
    
    for indx_t_c, t_c in enumerate(T_cond):
        interim_exp[t_c] = {}
        for indx_a_c, a_c in enumerate(a_cond):
            # e.g: a_cond = ['U', 'D'], action space, not strategy space
            # Fix one pure strategy, pi_n(t_n)
            for strgy_t in A_targ:
                # e.g. A_targ: = ['LL', 'LR', 'RL', 'RR'], strategy space, not action space
                # if len(T_cond) == 2:
                    # print('strgy_t', strgy_t)
                holder = []
                for indx_t_targ, a_t in enumerate(strgy_t):
                    # Over 'L', 'R' in 'LR', p(t_{-n}|t_n) * u_n(t_n, t_{-n}, a_n, pi_{-n}(t_{-n}) for one type t_{-n}
                    if '1' in cond_player:
                        holder.append(U_cond[indx_t_c][indx_a_c, a_targ.index(a_t)] * belief[t_c][T_targ[indx_t_targ]])
                    elif '2' in cond_player:
                        holder.append(U_cond[indx_t_c][a_targ.index(a_t), indx_a_c] * belief[t_c][T_targ[indx_t_targ]])
                    else:
                        raise ValueError('interim_exp_comp error')
                # Compute the interim expected utility for player n given its type t_n, and pure strategy pi_n(t_n) = a_n, and pi_{-n}
                interim_exp[t_c][(a_c, strgy_t)] = sum(holder)
    return None

In [19]:
def belief_construct(T_cond, T_targ, cond_prior, targ_prior):
    for t_cond in T_cond:
        belief[t_cond]={}
        for t_targ in T_targ:
            # p(t_targ|t_cond)
            belief[t_cond][t_targ]= targ_prior[t_targ]*cond_prior[t_cond] / cond_prior[t_cond]
    return None

In [20]:
interim_exp = {}
ex_ante_exp = {}
belief = {}
belief_construct(T1, T2, prior_1, prior_2)
interim_exp_comp('player 1', U1, T1, a1, a2, A2, T2)
belief_construct(T2, T1, prior_2, prior_1)
interim_exp_comp('player 2', U2, T2, a2, a1, A1, T1)

In [21]:
interim_exp

{'SH_1': {('U', 'LL'): 3.0000000000000004,
  ('U', 'LR'): 0.6000000000000001,
  ('U', 'RL'): 2.4000000000000004,
  ('U', 'RR'): 0.0,
  ('D', 'LL'): 2.0,
  ('D', 'LR'): 1.2000000000000002,
  ('D', 'RL'): 1.8,
  ('D', 'RR'): 1.0},
 'PD_2': {('L', 'U'): 3.0, ('L', 'D'): 1.0, ('R', 'U'): 4.0, ('R', 'D'): 2.0},
 'SH_2': {('L', 'U'): 3.0, ('L', 'D'): 0.0, ('R', 'U'): 2.0, ('R', 'D'): 1.0}}

In [22]:
belief

{'SH_1': {'PD_2': 0.2, 'SH_2': 0.8},
 'PD_2': {'SH_1': 1.0},
 'SH_2': {'SH_1': 1.0}}

In [91]:
game_instance = {
    'player 1': {
        'p_1': {
            'SH_1': 1.0
        },
        'A1': ['U', 'D'],
        'SH_1 u_1': np.array([[3,0], [2,1]]),
        'T1': ['SH_1']
    },
    'player 2': {
        'p_2': {
            'PD_2': 0.2,
            'SH_2': 0.8
        },
        'A2': ['L', 'R'],
        'PD_2 u_2': np.array([[3,4], [1,2]]),
        'SH_2 u_2': np.array([[3,2], [0,1]]),
        'T2': ['PD_2', 'SH_2']
    }
}

In [19]:
 # prior_list = [prior_1, prior_2]
 # for indx_T_cond, T_cond in enumerate([T1, T2]):
 #        prior_cond = prior_list[indx_T_cond]
 #        prior_targ = prior_list.remove(prior_cond)[0]
 #        T_targ = [T1, T2].remove(T_cond)
 #        belief_construct(T_cond, T_targ, prior_cond, prior_targ)

In [24]:
def two_type_strgy_name_gen(A=None, action_name=None, num_a=1):
    """
    Construct all pure strategies for one player
    :param A: strategy construct holder, e.g. A = ['LL', 'LR', 'RL', 'RR']
    :param num_a: number of actions in action space
    """
    B = []
    if A is not None:
        if action_name is None:
            for a in A:
                for action_indx in range(num_a):
                    B.append(f'{a}{action_indx}')
        else:
            for a in A:
                for action in action_name:
                  B.append(a+action)
    else:
        if action_name is None:
            assert num_a >= 1
            B = [f'{a_indx}' for a_indx in range(num_a)]
        else:
            assert type(action_name) is list
            B = action_name
    return B

In [21]:
# A = None
# action_name = ['L', 'R']
# num_a = len(action_name)

A = None
action_name = None
num_a = 2
# A = two_type_strgy_name_gen(A=A, action_name=action_name, num_a=num_a)

for count in range(3):
    A = two_type_strgy_name_gen(A=A, action_name=action_name, num_a=num_a)
    print('A', A)
# A
# AA = two_type_strgy_name_gen(A, action_name, num_a)
# AAA = two_type_strgy_name_gen(AA, action_name, num_a)
# AAA
# two_type_strgy_name_gen()

A ['0', '1']
A ['00', '01', '10', '11']
A ['000', '001', '010', '011', '100', '101', '110', '111']


In [111]:
import copy
def find_undominated_actions(U_in, i, A, DOPRINT=False):
    '''find_undominated_actions: finds the actions for player i that are
        not strictly dominated by another action
        
        INPUTS: 
            U_in: (matrix, na1*na2) Payoffs (player 1, player 2)
            i: (integer) Which player we are currently examining
            A: (list) List of actions (len = # of actions for this player)
            
        OUTPUT: 
            AA: (list) undominated actions 
            IA: (list of integers) integers i s.t. AA = [A[i] for i in IA]
            ANYDOMINATED: (bool) True if at least one action was strictly dominated
    '''
    
    AA = []
    IA = []
    nA = len(A)
    
    # 1. ensure that U has actions of player i along 0th dimension 
    if i == 0: 
        # 1.a already the case 
        U = np.copy(U_in)
    else: 
        # 1.b transpose 
        U = U_in.T 
    
    # 2. determine if each action has other dominated actions 
    for ia in range(nA): 
        DOMINATED = False 
                
        for ia_ in range(nA): 
            # 2.a loop through all *other* strategies 
            if ia_ == ia: 
                continue

            # 2.b check if ia_ always gives a higher payoff than ia (i.e. domination)
            if np.all(U[ia_] > U[ia]): 
                DOMINATED = True
                break # exit search: enough that we have found one 
        
        # 2.c append or not 
        if not DOMINATED: 
            AA.append(A[ia])
            IA.append(ia)
            
    # 3. convenient boolean 
    ANYDOMINATED = (len(AA) < len(A))
    
    return AA,IA,ANYDOMINATED


def IESDS(A, U, DOPRINT=False, maxit=10000): 
    '''Iterated Elimination of Strictly Dominated Strategies 
        INPUTS: 
            A: (list of lists) n lists (one for each player), 
                    each has len = # of pure strategies to player i (nai)
            U: (list, len=n) list of na1*na2 matrices of payoffs
            DOPRINT: (bool) whether to print output to terminal 
            maxit: (int) break algorithm if this count is ever reached
                (note: the algorithm is not approximate so we can compute 
                what maxit is in the worst case)
        OUTPUT: Actions and payoffs for the undominated game
            A_undominated: (n-list of vectors) 
            U_undominated: (n-list of matrices of payoffs)
    '''
    
    U_undominated = copy.copy(U)    # list of utility matrices for both players
    A_undominated = copy.copy(A)    # list of action spaces for both players
    
    n = len(U)  # n, number of players
    na1,na2 = U[0].shape    # nai, number of actions for player i

    # checks 
    assert n == 2, f'Code only implemented for 2-player games '
    assert len(A) == n
    for i in range(n): 
        assert len(A[i]) == U[i].shape[i]
        assert U[i].shape == (na1,na2), f'Payoff matrix for player {i+1} is {U[i].shape}, but {(na1,na2)} for player 1'

    # initialize flags 
    D = np.ones(n, dtype='bool')
    
    for it in range(maxit): 

        for i in range(n): # for every player
            # find undominated actions (A_undominated[i]) and the undominated action indices (IA), (D[i]) indicates
            # whether player i has strictly dominated actions
            A_undominated[i], IA, D[i] = find_undominated_actions(U_undominated[i], i, A_undominated[i], DOPRINT)
            print(f'D[{i}]', D[i])

            # if we found at least one, remove it/them from the game 
            if D[i]: 
                # remove from both players' payoff matrices 
                for j in range(n): # for every player
                    if i == 0: 
                        U_undominated[j] = U_undominated[j][IA, :]
                        print('U undominated[0]', U_undominated[j])
                    else: 
                        U_undominated[j] = U_undominated[j][:, IA]
                        print('U undominated[1]', U_undominated[j])


        # break once we have run an iteration without finding any strategies to remove 
        if D.any() == False: 
            break

    return A_undominated, U_undominated

In [116]:
print('t1_mod\n', t1_mod)
print('t2_mod\n', t2_mod)
A_, T_ = IESDS(A=[A1_mod, A2_mod], U=[t1_mod, t2_mod], DOPRINT=True)

D[0] False
D[1] True
U undominated[1] [[2.4 0. ]
 [1.8 1. ]]
U undominated[1] [[3.2 2.4]
 [0.4 1.2]]
D[0] False
D[1] False


In [138]:
class ABC:
    def __init__(self):
        self.vals = val()
    class val:
        def __init__(self):
            self.C = 3

In [139]:
val = ABC()

TypeError: 'ABC' object is not callable

In [142]:
class Color:

    # constructor method

    def __init__(self):

        # object attributes

        self.name = 'Green'
        self.lg = self.Lightgreen()

    def show(self):
        print('Name:', self.name)


    # create Inner Lightgreen class

    class Lightgreen:

        def __init__(self):
            self.name = 'Light Green'
            self.code = '024avc'

        def display(self):
            print('Name:', self.name)
            print('Code:', self.code)

# create Color class object
outer = Color()


# method calling
outer.show()

# create a Lightgreen
# inner class object

g = outer.lg

# inner class method calling

g.display()
outer.lg.name

Name: Green
Name: Light Green
Code: 024avc


'Light Green'